In [1]:
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import time
import sys
import os

import seaborn as sns
import gensim
from datetime import datetime, date

import django
import platform

if platform.node() == "srv-mcc-apsis":
    sys.path.append('/home/muef/tmv/BasicBrowser/')
else:
    # local paths
    sys.path.append('/Documents/Data/tmv/BasicBrowser/')

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

# import from appended path
import parliament.models as pm
from parliament.tasks import do_search, run_tm
import cities.models as cmodels
import scoping.models as sm
from django.contrib.auth.models import User
from tmv_app.models import *
from utils.tm_mgmt import update_topic_scores
from utils.text import *
from django.db.models import Q, Count, Func, F, Sum, Value, Case, When, IntegerField

from parliament.utils import merge_utterance_paragraphs

# find text for examples

In [4]:
# more than 2 mentions of coal + later than 2017
uts_sample = pm.Utterance.objects.filter(search_matches=99, document__date__gte=date(2017,10,1), speaker__party__name='cducsu')
print(uts_sample.count())

44


In [5]:
# print speeches that mention coal early on

p = re.compile("Kohle")

for ut in uts_sample:
    text = ut.paragraph_texts
    m = p.search(text)
    if m and m.start() < 1000:
        print(ut, ut.speaker, ut.speaker.party, ut.document.date)
        print(m.start())
        print(ut.paragraph_texts[:m.start()+100], "...")

Utterance object (3212624) Dr. Klaus-Peter Schulze CDUCSU 2018-03-23
379
Herr Präsident! Liebe Kolleginnen und Kollegen! Meine sehr verehrten Damen und Herren auf den Zuschauertribünen! Zunächst einmal herzlichen Glückwunsch, Frau Ministerin! Ich freue mich auf eine hoffentlich gute und sachliche Zusammenarbeit, und ich finde es gar nicht schlimm, wenn Sie Mitglied der IG BCE sind. Denn das ja schon mehrfach angesprochene Thema „Wie weiter mit der Kohle?“ und das Bestreben, in der Diskussion zu einem Endpunkt zu kommen, werden es notwendig machen ...
Utterance object (3363455) Dr. Joachim Pfeiffer CDUCSU 2017-11-21
195
Frau Präsidentin! Meine sehr geehrten Damen und Herren! Wie ist die Lage? Der Weltmarkt für konventionelle Stromerzeugung verändert sich rasant. Das ist nicht neu. Wir haben es beim Übergang von Kohle zu Öl erlebt, wir haben es bei der Kernenergie erlebt, wir erleben es bei der Steinkohle, bei  ...
Utterance object (3368431) Dr. Klaus-Peter Schulze CDUCSU 2018-03-23
379


In [14]:
# print speeches with search term

uts_sample = pm.Utterance.objects.filter(search_matches=89,
                                         document__date__gte=date(1998,1,1),
                                         document__date__lte=date(2005,12,31))
print("searching in {} utterances".format(uts_sample.count()))

p = re.compile("coemission")

for ut in uts_sample:
    text = ut.paragraph_texts
    m = p.search(text)
    if m:
        print(ut, ut.speaker, ut.speaker.party, ut.document.date)
        print(ut.paragraph_texts)
        print("")

searching in 886 utterances


In [28]:
# search texts scoring high in a particular topic

run_id = 2350
topic_title = 'climate protection'
topic = Topic.objects.get(title=topic_title, run_id=run_id)

doctopics = pm.Utterance.objects.filter(
    doctopic__topic=topic,
    doctopic__run_id=run_id,
    doctopic__score__gt=0.01,
    document__date__gte=date(2019,1,1)
)

p = re.compile("Arbeit")

for ut in doctopics.order_by('-doctopic__score'):
    text = ut.paragraph_texts
    m = p.search(text)
    if m:
        print(ut, "protocol {}/{},".format(ut.document.parlperiod.n, ut.document.sitting), ut.document.date)
        print("{} ({}):".format(ut.speaker, ut.speaker.party))
        print(ut.paragraph_texts)
        print("")

Utterance object (3382184) protocol 19/77, 2019-01-31
Michael Theurer (FDP):
Sehr geehrter Herr Präsident! Liebe Kolleginnen und Kollegen! Wie richtig es ist, dass die Freien Demokraten diese Aktuelle Stunde beantragt haben, zeigt der Verlauf der Debatte; denn es hat sich herausgestellt, welche immensen Bewertungsunterschiede es zwischen Union einerseits und SPD andererseits gibt. Da kann man nicht feststellen, dass die Große Koalition einig wäre. Im Gegenteil, wir dürfen gespannt sein, was am Ende rauskommt.
Herr Kollege Krischer, wenn Sie sich heute hierhinstellen und vom Prinzip Lindner sprechen, sage ich mal, was Ihr Prinzip ist. Ihr Prinzip Krischer ist: Fakten biegen, bis die Balken brechen. Hier zu behaupten, dass der Kohlestromanteil verantwortlich dafür sei, dass die Bundesrepublik Deutschland die Klimaziele 2020 nicht erreicht, ist, grob gesagt, die Unwahrheit. Das sage nicht ich, sondern das steht im Bericht der Kohlekommission. Ich zitiere mit Erlaubnis des Präsidenten die 

Utterance object (3390590) protocol 19/108, 2019-06-28
Lorenz Gösta Beutin (LINKE):
Frau Präsidentin! Liebe Kolleginnen und Kollegen! Liebe Zuschauerinnen und Zuschauer! Die Bundestagsfraktion Die Linke bringt heute den Antrag ein, den Klimanotstand in Deutschland anzuerkennen und gleichzeitig endlich ein Klimaschutzgesetz vorzulegen, das geeignet ist, den Beitrag Deutschlands an der Erfüllung des Pariser Klimaabkommens einzuhalten. Genau das ist es, was wir jetzt brauchen. Das ist nicht irgendwie illusorisch oder sonst irgendwas. Vielmehr haben wir mittlerweile die Situation, dass fast 700 Städte weltweit den Klimanotstand ausgerufen haben. Wir haben die Situation, dass Kanada, Großbritannien und Irland den Klimanotstand ausgerufen haben. Gestern ist Frankreich dazugekommen. Das sind doch keine Pillepallestaaten. Gerade in diesem Augenblick umzingelt Fridays for Future den Deutschen Bundestag. Die jungen Leute sagen uns: Wir mahnen euch. Wir mahnen euch seit dem November des letzten J

Utterance object (3382922) protocol 19/80, 2019-02-14
Uwe Kamann (AFD):
Sehr geehrte Frau Präsidentin! Liebe Kollegen! Werte Besucher! Einige Anträge zur heutigen Lesung in Sachen Kohleausstieg verdeutlichen, woran die Politik in diesem Lande krankt: an starrsinnigen Standpunkten und der mangelnden Nutzung des gesunden Menschenverstandes. Die AfD fordert kategorisch den Ausstieg aus dem Kohleausstieg. Linke und Grüne haben konzeptionell leider wirklich nichts zu bieten. Die Linke will nur eines: den Kohleausstieg schnell und sozial gerecht umsetzen. Kein Hinterfragen, keine Verantwortung, keine wirtschaftliche Folgeabschätzung! Aber als ehemaliger Bergmann schätze ich durchaus, dass Die Linke an die Arbeiter denkt.
Doch ich sehe auch, dass die Arbeitsplatzbeschaffung am grünen Tisch geplant wird. Der Wandel – das ist ein kapitaler Fehler – erfolgt nicht durch eine gesunde marktwirtschaftliche Entwicklung, sondern nach planwirtschaftlicher Vorgabe. Das hat schon bei -Honecker und Ulbric

Jens Koeppen (CDUCSU):
Herr Präsident! Liebe Kolleginnen und Kollegen! Wer sich diese Debatte angeschaut hat, konnte feststellen: Es herrscht Einigkeit in diesem Plenum, und zwar in zwei Punkten:
Erster Punkt. Einigkeit besteht darüber, dass die größte oder eine der größten Herausforderungen in dieser Zeit die massive globale Ressourcenverschwendung ist, der massive Eingriff in die Umwelt, in die Natur.
Zweiter Punkt. Wir sind uns einig darüber, dass es gut ist, wenn sich junge Menschen engagieren, wenn sie voller Empathie, voller Begeisterung und voller Hilfsbereitschaft für eine lebenswerte Zukunft kämpfen und streiten. Das ist unbestritten. Aber dafür brauchen wir nicht Greta; denn das haben die jungen Leute schon immer gemacht, jedenfalls die, die ich kenne, in den Wahlkreisen, in den Vereinen, in den Schulen, beim THW, bei der Feuerwehr, bei den Jugendorganisationen. Darauf können wir stolz sein; denn das ist der Kitt, der die Gesellschaft zusammenhält. Nicht einig sind wir uns in

In [53]:
pm.Utterance.objects.get(id=3363455).paragraph_texts

'Frau Präsidentin! Meine sehr geehrten Damen und Herren! Wie ist die Lage? Der Weltmarkt für konventionelle Stromerzeugung verändert sich rasant. Das ist nicht neu. Wir haben es beim Übergang von Kohle zu Öl erlebt, wir haben es bei der Kernenergie erlebt, wir erleben es bei der Steinkohle, bei der Braunkohle und jetzt auch bei Gasturbinen.\nEs ist im Übrigen auch politisch gewollt, auch von Ihnen, Herr Schulz, und Ihrer Partei, dass die Erneuerbaren stärker wachsen und ihr Anteil ausgeweitet wird. Deshalb ist in Europa heute kein einziges konventionelles Kraftwerk mehr im Bau. Weltweit sieht es noch anders aus, aber die Erneuerbaren haben hier stark aufgeholt.\nDerzeit werden 110 bis 120 Gasturbinen – und um die geht es – pro Jahr auf dem Weltmarkt benötigt, in Zukunft eher weniger. Weltweit gibt es Kapazitäten, 400 Turbinen pro Jahr herzustellen. Darauf muss man reagieren. Das gilt auch für ein Unternehmen wie Siemens, die Gott sei Dank bei der Windkraft, onshore wie offshore, Weltma